<font size="2">$\textbf{Problem 1: Coding, integration, and a simple econ example}$</font>

A profit-maximizing firm faces a demand curve given by $P(q) = a - bq$  
where $b\sim logN(\mu,\sigma)$ and has a cost function given by $C(q) = cq$.

<br>
$\text{1. Solve for the optimal quantity analytically}$

$\pi(q) = P(q)\cdot q - C(q) = (a - bq)\cdot q - cq \ \ \Rightarrow \ E[\pi(q)] = (a - E[b]q)\cdot q - cq$

F.O.C. (MR = MC): $a - 2E[b]q = c$.  
S.O.C: $-2E[b]<0$. (Satisfied) 

Therefore, $q^* = \frac{a - c}{2E[b]}$ where $E[b] = e^{\mu +\sigma^2/2}$

<br>
$\text{2. Write a function called profit_max_q}(a, c, \mu, \sigma, method, n)$

that returns the numerical optimal quantity given a set of inputs $(a, c, \mu, \sigma, method, n)$, where method is a string  
that takes on a value of "mc" or "quad" and determines whether you integrate using Monte Carlo or quadrature methods,  
and n is the number of Monte Carlo draws or quadrature nodes.

In [1]:
using Distributions, CompEcon

In [2]:
#=
    Function profit_max_q(a, c, mu, sigma, method, n) 
    Inputs:
    a:     Intercept of demand curve.
    c:     Slope of demand curve.
    mu:    Mean of lognormal distribution.
    sigma: Standard deviation of lognormal distribution.
    method:Integration method. Type-string. method="mc": Monte Carlo Simulation and method="quad": Quadrature.
    n:     number of monte Carlo draws or quadrature nodes
    Output:Optimal quantity
=#

function profit_max_q(a, c, mu, sigma, method, n)
    if method == "mc"                   # method monte-carlo
        x = rand(LogNormal(mu,sigma),n) # generating n random numbers from lognormal
        Eb = sum(x)/n                   # Expected value of b         
        return (a - c) / (2*Eb)         # optimal value of quantity
        
        elseif method == "quad"         # method quadrature
        if n > 198                      # check the number of nodes; if n > 198, we get error
            n = 198                     # fix n = 198 in that case
        end
        (x,w) = qnwlogn(n,mu,sigma^2)   # generating n quadrature nodes and weights
        Eb = sum(w .* x)                # Expected value of b
        return (a - c) / (2*Eb)         # Optimal value of quantity
        
    else
        print("Method Error")           # method error other than mc and quad
    end
end

profit_max_q (generic function with 1 method)

$\text{3. Use profit_max_q to solve the problem}$

Choose a set of values $(a, c, \mu, \sigma)$ and use profit_max_q to solve the problem for both approaches to integration. Use the CompEcon package to implement the quadrature routine.

In [3]:
# example of parameter values with monte-carlo integration
a = 1000
c = 1
mu = 0
sigma = 1
method = "mc"
n = 10000

@time q_mc = profit_max_q(a,c,mu,sigma,method,n)

  1.352627 seconds (3.86 M allocations: 191.145 MiB, 5.80% gc time)


298.40943890975825

In [4]:
# change the method to quadrature integration
method = "quad"

@time q_quad = profit_max_q(a,c,mu,sigma,method,n)

  0.575159 seconds (1.53 M allocations: 80.472 MiB, 4.02% gc time)


302.96206452646123

In [5]:
# the value of the analytical solution
q_an = (a - c) / (2* exp(mu + sigma^2/2))

print("error in mc method is:")
println(abs(q_mc - q_an))
print("error in quad method is:")
print(abs(q_quad - q_an))

error in mc method is:4.55262561670213
error in quad method is:8.526512829121202e-13

<font size="2">$\textbf{Problem 2: Coding and Monte Carlo}$</font>

Approximate $\pi$ using Monte Carlo. You may only use rand() to generate random numbers. Here is how to think about approximating $\pi$:

1. Suppose $U$ is a two dimensional random variable on the unit square $[0,1]\times[0,1]$. The probability that $U$ is in a subset $B$ of  $(0,1)\times(0,1)$ is equal to the area of $B$.
2. If $u_1,...,u_n$ are iid draws from $U$, then as $n$ grows (by an LLN type argument), the fraction that falls inside $B$ is the probability of another iid draw coming from $B$.
3. The area of a circle is given by $\pi \times radius^2$.

In [6]:
n = 10^6                    # number of monte carlo draws

count = 0                   # Initialize count
for i = 1:n
    (x,y) = rand(2)         # two dimensional random number
    # check if point is within the unit circle, update the count.
    count = (x - 0.5)^2 + (y - 0.5)^2 < 1/4 ? count + 1 : count 
end

@time pi_approx = 4*count/n # Area=4$\pi$.

  0.000005 seconds (7 allocations: 240 bytes)


3.142224

<font size="2">$\textbf{Problem 4: Memory location¶}$</font>

Let's learn about some of the nuances of memory allocation.

$\text{1. Generate one 20000 x 20000 array of random numbers named x}$

$\text{2. Make a function called exp_cols}$  
which exponentiates the elements of x column by column (i.e. by broadcasting exp.()) and returns the exponentiated array.

$\text{3. Make a function called exp_rows}$  
which exponentiates the elements of x row by row (i.e. by broadcasting exp.()) and returns the exponentiated array.

In [7]:
x = rand(20000,20000)          # Array of random numbers

#=
Function exp_cols(x) 
Inputs:
x: Array.
Method: Exponential of the elements of array column by column
Output: 
y: Exponential of array
=#

function exp_cols(x)
    n = size(x,2)              # number of columns of the array
    y = similar(x)             # initialize the output array
    for i = 1:n        
        y[:,i] = exp.(x[:,i])  # exponents of elements column-wise
    end
    return y                   # exponent array
end

#=
Function exp_rows(x) 
Inputs:
x: Array.
Method: Exponential of the elements of array row by row
Output: 
y: Exponential of array
=#

function exp_rows(x)
    n = size(x,1)              # number of rows of the array
    y = similar(x)             # initialize the output array
    for i = 1:n
        y[i,:] = exp.(x[i,:])  # exponents of elements row-wise
    end
    return y                   # exponent array
end

exp_rows (generic function with 1 method)

$\text{4. Call exp_cols(x) and exp_rows(x) twice}$  
and calculate the elapsed time on the second call (avoids fixed cost of initial compiliation).

$\text{5. Is one faster than the other?}$

In [8]:
@time exp_cols(x);

  9.690325 seconds (194.16 k allocations: 8.949 GiB, 9.57% gc time)


In [9]:
@time exp_cols(x);

 12.320929 seconds (80.01 k allocations: 8.944 GiB, 6.68% gc time)


In [10]:
@time exp_rows(x);

 36.067744 seconds (226.25 k allocations: 8.951 GiB, 3.09% gc time)


In [11]:
@time exp_rows(x);

 26.514937 seconds (80.01 k allocations: 8.944 GiB, 3.64% gc time)


exp_cols() is faster than exp_rows()